### GaPFlow Integration Notes

- `problem` class contains all necessary information
 - 'move out' explicit-solver features to keep `problem` generic
- muGrid features can be utilized to compute and hold quadrature values

&rarr; should be possible to keep structure

Required additions:
- rewrite models using `jnp`
- extend all model classes to include auto-diff functions
- extend FEM `pre_run()` - create auto-diff functions, init terms
- extend FEM `update(mode)`
  - update model outputs (eos, bulk stress, wall stress, temperature)
  - update quadrature point values
  - update auto-diff function values
  - ready to compute M, R
  - in steady state, do either one iteration or whole process (makes more sense e.g. for scipy minimizers)
  - in dynamic mode do one time step
  - keep flexible usage, so that we can do `run(steady)` followed by `run(dynamic)`
  - keep flexible term/model usage (even within one problem instance?)
- `Energy()` class corresponding to `Pressure()`
  - Temperature (heat capacity $c_v(T)$, thermal conductivity $k(T, \rho, \alpha)$), vaporization energy $\Delta U_{\mathrm{vap}}(T)$ and wall heat exchange models

New files:
- `explicit_solver.py`
- `FEM_solver.py`
- `terms.py` - Residual terms
- `solvers.py` - Newton, CG, TrustRegion, GMRES, ...

New input entries:
- `solver`
  - 'explicit'
  - 'FEM'
- `FEMsolver`
  - `solver_type` (newton_alpha, scipy_trust_region, ...)
  - `rtol`, `ftol`, `maxiter`, `alpha` (solver-dependent)
  - `callbacks`: cavitation_check, ...
  - `equations`
    - `energy`: bool (user-friendly high-level setting)
    - `term_list` (manually selecting terms to include)
- `grid`
  - `ic` (specify arbitrary initital condition)


Open questions:
- how include auto-diff functions only when using FEM
  - building auto-diffs from central point or each in `Pressure`, `WallStress`, etc.
  - Option 1: Building auto-diffs AND terms in `pre_run()`
  - Option 2: Building auto-diffs at model instantiation and only build terms in `pre_run()`

&rarr; use option 1: no unnecessary `if's` in model classes but all centrally initialized from `pre_run()` in `FEM_solver.py`